## Table of Contents
#### [1. Setup](#Setup)
#### [2. Problem statement](#problem_statement)
#### [3. Create propagating function](#create_propagating_function)
#### [4. Epistemic uncertainty](#propagate_epistemic_uncertainty)
#### <small>[4.1. Uncertainty Characterisation](#4.1-uncertainty-characterisation)</small>
#### <small>[4.2. Uncertainty propagation methods](#4.2-uncertainty-propatation-methods)</small>
##### <small>[4.2.1. Endpoints propagation](#4.2.1-Endpoints-propatation)</small>
##### <small>[4.2.2. Subinterval reconstitution propagation](#4.2.2.-Subinterval-reconstitution-propagation)</small>
##### <small>[4.2.3. Sampling propagation](#4.2.3.-Sampling-propagation)</small>
##### <small>[4.2.4. Optimisation methods](#4.2.3.-Optimisation-methods)</small>
#### [5. Aleatory uncertainty](#aleatory_uncertainty_propagation)
#### [5. Mixed types of uncertainty](#mixed_uncertainty_propagation)

***
<a id="setup"></a>
# 1. Setup

Import the libraries

In [1]:
import numpy as np
from PyUncertainNumber.propagation.uncertaintyPropagation import Propagation
from PyUncertainNumber.characterisation.uncertainNumber import UncertainNumber as UN
import docutils # how to use this????

C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\pba\interval.py:69: SyntaxWarning: invalid escape sequence '\i'
  """
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\pba\interval.py:791: SyntaxWarning: invalid escape sequence '\m'
  """
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\pba\interval.py:805: SyntaxWarning: invalid escape sequence '\m'
  """
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\pba\interval.py:823: SyntaxWarning: invalid escape sequence '\m'
  """Returns the midpoint of the interval
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\pba\interval.py:907: SyntaxWarning: invalid escape sequence '\m'
  """
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncertainNumber\characterisation\utils.py:140: SyntaxWarning: invalid escape sequence '\d'
  pctg = re.findall("\d*%", txt)
C:\Users\Ioanna\Documents\GitHub\PyUncertainNumber\src\PyUncerta

AttributeError: type object 'Pbox' has no attribute 'STEPS'

In [4]:
%load_ext autoreload
%autoreload 2
%load_ext rich

In [5]:
# import user-defined function

USER_dir = 'C:\\Users\\Ioanna\\Documents\\GitHub\\PyUncertainNumber\\Michell_truss'
import sys
sys.path.append(USER_dir)

<a id="problem_statement"></a>
# 2. Problem statement

We want to design a lightweight mounting structure to support an electric motor to the main spar of a wing. Despite the best intentions of the design team to achieve minimum weight for the structure, considerable uncertainty surrounds its main sizing parameters, giving rise to the risk of producing a lightweight structure which may require substantial changes later on in the design process.

The motor mount is a “cage”, each of whose four sides is a Michell lattice, such as the one shown in Fig.1.The complex geometry of each lattice can be created with relative ease through the principles of procedural geometry generation, where topological features can be implemented as variables just as easily as physical dimensions. In particular, the geometry of each lattice is created by the algorithm proposed by Chan {REF}, which implements the mechanical principles developed by Michell {REF}. The complete study can be found in Ioannou I, Hristov PO, Yong HK, Marsh R, Silva E, Sobester A, Ferson S. Towards a Framework for Non-intrusive Uncertainty Propagation in the Preliminary Design of Aircraft Systems. In AIAA Scitech 2023 Forum 2023 (p. 2373). 

Grasshopper can not be called externally through python. Nonetheless, our code is still useful and can be used to provide necessary input combinations for methods other than optimisation. 

![alt text](<Motor_mount2_7_24Divs.png>)!

**Fig.1** Two instances of the studied parametric motor mount.

***
<a id="propagate_epistemic_uncertainty"></a>
# 4. Epistemic uncertainty

<a id="3.1.-uncertainty-characterisation"></a>

## 3.1. Uncertainty Characterisation

Construct `UncertainNumbers` objects for the inputs assuming they are intervals with the lower and upper values are seen below


Characterise the uncertainty of the input uncertain numbers:

- $D$ Number of divisions. 
- $R_{out}$ The ratio of between the width of the outer beams (black) and the overall length $L$ of the lattice.
- $R_{in}$ The ratio between the width of the inner beams of the lattice (white), excluding the second beams, and the width of the outer beams.
- $R_{in2}$ The ratio between the width of the second inner beams (green) and the width of the outer beams
- $F$ the download force applied to the tip of the lattice.

![alt text](<model_params.PNG>)
**Fig.2** Geometric parameterisation of a lattice with 4 divisions.

In [6]:
D = UN(name='divisions', symbol='D', essence='interval', bounds=[4, 24]) 
R_out = UN(name='outer_length_ratio', symbol='R_out', essence='interval', bounds= [0.002, 0.008])
R_in = UN(name='inner_outer_ratio', symbol='R_in', essence='interval', bounds= [0.25, 1.00])
R_in2 = UN(name='inner2_outer_ratio', symbol='R_in2', essence='interval', bounds=[0.25, 1.00])
F = UN(name='vertical force', symbol='F', units='kN', essence='interval', bounds= [15, 25])


In [ ]:
print(D)

<a id="3.2-uncertainty-propatation-methods"></a>

## 3.2 Ucertainty propagation methods 

Choose from a suite of black box propagating techniques to propagate the intervals through the model. 

<a id="3.2.1.-Endpoints-propatation"></a>

### 3.2.1. Endpoint propagation

The endpoint propagation method (Dong and Shah, 1987) is a straightforward way to project intervals through the code, by projecting all input combinations produced by the Cartesian product of the interval bounds. This results in a total of $n = 2^{d}$. 

In [ ]:
2**5

In [ ]:
METHOD = "endpoints"

a = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
a.raw_data['x'] # the input combinations.

In [ ]:
a.print() # display of results
#TODO perhaps change the display of hte results only the input and output to appear. 

<a id="3.2.2.-Subinterval-reconstitution"></a>

### 3.2.2. Subinterval reconstitution

In [ ]:
METHOD = "subinterval"

a = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None,
          n_sub = 3, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
a.raw_data['x'] # the input combinations.

In [ ]:
a.print()

<a id="3.2.3.-Sampling-propatation"></a>

### 3.2.3. Sampling propagation

In [ ]:
METHOD = "monte_carlo"

a = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None,
          n_sam = 300, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
METHOD = "monte_carlo_endpoints"

b = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None,
          n_sam = 300, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
#a.print()
len(a.raw_data['x'])

In [ ]:
len(b.raw_data['x'])

In [ ]:
METHOD = "endpoints_cauchy"

c = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None,
          n_sam = 300, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
c.raw_data['x']

In [ ]:
print(len(c.raw_data['K']))
c.raw_data['K']

<a id="3.2.5.-Optimisation_methods"></a>

### 3.2.5. Optimisation propagation

In [ ]:
METHOD = "local_optimisation"

a = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )

In [ ]:
METHOD = "genetic_optimisation"

a = Propagation(vars=[D, R_out, R_in, R_in2, F], 
          fun=None, 
          method = METHOD, 
          save_raw_data = "yes", 
          base_path = USER_dir
         )